In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors, word2vec
import gensim
import gensim.downloader
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from scipy.spatial.distance import cosine as cosDist
import numpy as np
import pandas as pd
import json
import re
import os

In [8]:
os.chdir("Downloads")

In [ ]:
def setModel(Model):

In [9]:
global w2vModel

w2vModel = word2vec.KeyedVectors.load_word2vec_format("word2vec-google-news-300.gz",binary=True)

    #path = get_tmpfile('word2vec.model')

In [10]:
sentences, target =	(
  [
    "The earthquake caused severe damage to 135 houses in Mirpur District, with a further 319 being partially damaged, most in Mirpur and just four in Bhimber District.",
    " Two bridges were reported damaged and parts of several roads were affected, particularly 14 km of the Main Jatlan Road.",
    " According to the chairman of Pakistan's National Disaster Management Authority (NDMA), 'In Mirpur, besides the city, a small town Jatlan, and two small villages Manda and Afzalpur' were among the worst-hit areas.",
    " According to him, the main road which runs alongside a river from Mangla to Jatla suffered major damage.",
    " According to the officials, the Mangla Dam, Pakistan's major water reservoir, was spared.",
    " However, the dam's power house was closed, which resulted in a loss of 900 megawatts to Pakistan's national power grid.",
    " At 7:20 pm, power generation at Mangla was resumed, restoring 700 MW to the national grid.",
  ],
  "As per the administrator of Pakistan's National Disaster Management Authority (NDMA), 'In Mirpur, other than the city, a community Jatlan, and two little towns Manda and Afzalpur' were among the most exceedingly awful hit regions."
)

In [3]:
def findPlagiarism(sentences, target):

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    target = re.sub(r'[^a-z0-9\s]', '', target.lower())
    target = target.split()
    print(target)
    target_words = []
    target_words = [lemmatizer.lemmatize(word) for word in target if word not in stop_words]
    #words = set(words)

    target_vectors = [w2vModel[word] for word in target_words if word in w2vModel]
    target_vector = np.mean(target_vectors, axis = 0)

    #print(target_vector)

    words = []
    cosine_distance = []
    for sentence in sentences:
        #print(sentence)
        sentence = re.sub(r'[^a-z0-9\s]', '', sentence.lower())
        sentence = sentence.split()
        words = [lemmatizer.lemmatize(word) for word in sentence if word not in stop_words]
        #print(words)
        vectors = [w2vModel[word] for word in words if word in w2vModel]
        avg_vector = np.mean(vectors, axis = 0)
        #print(avg_vector)
        cosine = cosDist(target_vector, avg_vector)
        #print(cosine)
        cosine_distance.append(cosine)

    target_index = np.argmin(cosine_distance)

    #print(target_index)
    return target_index

In [11]:
findPlagiarism(sentences, target)

['as', 'per', 'the', 'administrator', 'of', 'pakistans', 'national', 'disaster', 'management', 'authority', 'ndma', 'in', 'mirpur', 'other', 'than', 'the', 'city', 'a', 'community', 'jatlan', 'and', 'two', 'little', 'towns', 'manda', 'and', 'afzalpur', 'were', 'among', 'the', 'most', 'exceedingly', 'awful', 'hit', 'regions']


2

In [ ]:
def classifySubreddit_train(file):

    global lr_w2v
    global Scaler
    global le
    lr = LogisticRegression(penalty = 'l2', solver = 'sag', max_iter = 2000)

    #file = 'redditComments_train.jsonlist'
    with open(file,  encoding='utf-8') as json_file:
        training_set = [json.loads(line) for line in json_file]

    train_class = [line['subreddit'] for line in training_set]
    train_comments = [line['body'] for line in training_set]

    #pd.Series(subreddit_class).value_counts()

    x_train = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    url_pattern = r'\((https?://[^\s]+)\)'
    replacement_rules = {'“': '"', '”': '"', '’': "'", '--': ','}
    #replacements = lambda text: ''.join(replacement_rules.get(char, char) for char in text)
    for comment in train_comments:
        for char, replacement in replacement_rules.items():
            comment = comment.replace(char, replacement)
        #print('\nnew comment line:', comment)
        #for word in words:
        #if re.match(r'http[s]?://', comment):
        #Extract urls and add to token list. Text has useful urls for training
        combined_tokens = []
        urls = re.findall(url_pattern, comment)
        for url in urls:
            comment = re.sub(r'https?:\/\/(www\.)?', ' ', comment)
            sub_domain = url.split('/')
            domain_name = sub_domain[0]
            path_words = sub_domain[1:] if len(sub_domain) > 1 else []
            #Split domain and rest of url. Domain will likely be class specific
            split_domain = domain_name.split('.')
            path_tokens = []
            for i in path_words:
                path_tokens.extend(re.split(r'[\/\-_+\)\(\.]', i))            
            split_url = split_domain + [token for token in path_tokens if token.strip()]
            #Take care of edge cases
            for j in split_url:
                if ' ' in j:
                    combined_tokens.extend(j.lower().split())
                else:
                    combined_tokens.append(j.lower())
            for word in combined_tokens:
                word = re.sub(r'\W', '', word.lower())
        #Remove processed urls or else duplicates
        comment = re.sub(url_pattern, '', comment)
        words = re.findall(r'\b\w+(?:\'\w+)?\b', comment)
        cleaned_words = []
        for word in words:
            if word not in combined_tokens:
                if ' ' in word:
                    cleaned_words.extend(word.lower().split())
                else:
                    cleaned_words.append(word.lower())
        #print('CLEANED WORDS:', cleaned_words)
        combined_tokens.extend(cleaned_words)
        #combined_tokens = set(combined_tokens)
        #print('Combined tokens:', combined_tokens)
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in combined_tokens if token not in stop_words]
        #print('all tokens:', lemmatized_tokens)
        comment_vectors = [w2vModel[token] for token in lemmatized_tokens if token in w2vModel]
        x_train.append(np.mean(comment_vectors, axis = 0) if comment_vectors else np.zeros(w2vModel.vector_size))

    x_train = np.array(x_train)

    y = np.array(train_class)

    le = LabelEncoder()
    y_train = le.fit_transform(y)

    Scaler = MinMaxScaler()
    x_train_scaled = Scaler.fit_transform(x_train)

    lr_w2v = lr.fit(x_train_scaled, y_train)
    return lr_w2v, x_train_scaled

In [12]:
file = 'redditComments_train.jsonlist'

In [13]:
classifySubreddit_train(file)

(LogisticRegression(max_iter=2000),
 array([[0.54297067, 0.48082819, 0.47993328, ..., 0.50062613, 0.52656252,
         0.39240815],
        [0.5234093 , 0.4801517 , 0.45260427, ..., 0.47301351, 0.49185644,
         0.45293992],
        [0.55963307, 0.38940024, 0.43514474, ..., 0.55232955, 0.50502742,
         0.3720898 ],
        ...,
        [0.54166667, 0.41122328, 0.39849141, ..., 0.48622238, 0.55550874,
         0.45559251],
        [0.59729042, 0.49879816, 0.44997485, ..., 0.43743399, 0.54507289,
         0.37392396],
        [0.56649368, 0.45237159, 0.44338388, ..., 0.46795909, 0.52390996,
         0.48116131]]))

In [21]:
def classifySubreddit_test(text):
    with open(text,  encoding='utf-8') as json_file:
        test_set = [json.loads(line) for line in json_file]
        x_test = []

        subreddit_class_test = [line['subreddit'] for line in test_set ]
        comments_test = [line['body'] for line in test_set]
    for comment in comments_test:
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
        url_pattern = r'\((https?://[^\s]+)\)'
        replacement_rules = {'“': '"', '”': '"', '’': "'", '--': ','}
        #replacements = lambda text: ''.join(replacement_rules.get(char, char) for char in text)
        #comments = list(map(replacements, comments))
        #print('\nnew comment line:', comment)
        for char, replacement in replacement_rules.items():
            comment = comment.replace(char, replacement)
        #for word in words:
        #if re.match(r'http[s]?://', comment):
        #Extract urls and add to token list. Text has useful urls for training
        combined_tokens = []
        urls = re.findall(url_pattern, comment)
        for url in urls:
            comment = re.sub(r'https?:\/\/(www\.)?', ' ', comment)
            sub_domain = url.split('/')
            domain_name = sub_domain[0]
            path_words = sub_domain[1:] if len(sub_domain) > 1 else []
            #Split domain and rest of url. Domain will likely be class specific
            split_domain = domain_name.split('.')
            path_tokens = []
            for i in path_words:
                path_tokens.extend(re.split(r'[\/\-_+\)\(\.]', i))            
            split_url = split_domain + [token for token in path_tokens if token.strip()]
            #Take care of edge cases
            for j in split_url:
                if ' ' in j:
                    combined_tokens.extend(j.lower().split())
                else:
                    combined_tokens.append(j.lower())
            #Remove stray punctuation
            for word in combined_tokens:
                word = re.sub(r'\W', '', word.lower())
        #Remove processed urls or else duplicates
        comment = re.sub(url_pattern, '', comment)
        words = re.findall(r'\b\w+(?:\'\w+)?\b', comment)
        cleaned_words = []
        for word in words:
            if word not in combined_tokens:
                if ' ' in word:
                    cleaned_words.extend(word.lower().split())
                else:
                    cleaned_words.append(word.lower())
        combined_tokens.extend(cleaned_words)
        #combined_tokens = set(combined_tokens)
        #print('Combined tokens:', combined_tokens)
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in combined_tokens if token not in stop_words]
        #print('all tokens:', lemmatized_tokens)
        comment_vectors = [w2vModel[token] for token in lemmatized_tokens if token in w2vModel]
        x_test.append(np.mean(comment_vectors, axis = 0) if comment_vectors else np.zeros(w2vModel.vector_size))
    
        x_test = np.array(x_test)
        x_test_scaled = Scaler.transform(x_test)
    
        prediction = lr_w2v.predict(x_test_scaled)
        #lr_w2v.predict_proba(x_test)
    
        predicted_class = le.inverse_transform(prediction)
        print(predicted_class[0])
        return predicted_class[0]


In [22]:
text = "redditComments_test_notGraded.jsonlist"

In [23]:
classifySubreddit_test(text)

newToTheNavy


'newToTheNavy'